<a href="https://colab.research.google.com/github/ajayshivhare-aj/Product_Recommendation-for-online-retail/blob/main/OnlineRetail_Preference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Overview**


### *Customers are important to the survival and success of any business because they are the source of the revenue. The success of a business is the ability to satisfy customers and make them happy, and therefore turn a profit from them. Giving the customer recommendation about items that can buy it may attract them. Using retail transactional data can help in products recommendations.*

###*In this project, I will analyze an online retail transactional dataset from UCI ML to find the top preferences products, the preferences products for each active customers, the top preference products in each country and the top preference products in each month. So, I will answer the following questions:*


* ***What is the preference product for each active customer?***
* ***What is the top three preference products in each country?***
* ***What is the top three preference products in each month?***


###*Answering these questions can help in identifying the customers’ favorite products depending on the month and country.*

# **Dataset**


### **Online Retail.xlsx** - *The file contains all transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail. In the dataset, there are eight columns:*

### **InvoiceNo:** Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation.

### **StockCode:** Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.

### **Description:** Product (item) name. Nominal.

### **Quantity:** The quantities of each product (item) per transaction. Numeric.

### **InvoiceDate:** Invice Date and time. Numeric, the day and time when each transaction was generated.

### **UnitPrice:** Unit price. Numeric, Product price per unit in sterling.

### **CustomerID:** Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.

### **Country:** Country name. Nominal, the name of the country where each customer resides.


##***`The project is organized as follows `***

###***`Section 1 explores the data. Section 2 includes data preperation and Section 3 data analysis. Finally, Summary in the End.`***


## **1. Data Understanding**

### This step explores the dataset using different functions such as `head(), shape, describe()` and checks the null values using `isnull()`.

In [1]:
# Import needed libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re

pd.options.display.max_rows = None

In [2]:
# Read data in the excel file

df = pd.read_excel('/OnlineRetail.xlsx')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [3]:
df.shape

(541909, 8)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [5]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [6]:
# Check null values
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [7]:
# Check number of unique values
df.nunique()

InvoiceNo      25900
StockCode       4070
Description     4223
Quantity         722
InvoiceDate    23260
UnitPrice       1630
CustomerID      4372
Country           38
dtype: int64

In [8]:
# Check each stock code has only one description
df.groupby('StockCode')['Description'].unique().tail()

StockCode
gift_0001_20    [Dotcomgiftshop Gift Voucher £20.00, to push o...
gift_0001_30            [Dotcomgiftshop Gift Voucher £30.00, nan]
gift_0001_40                 [Dotcomgiftshop Gift Voucher £40.00]
gift_0001_50                 [Dotcomgiftshop Gift Voucher £50.00]
m                                                        [Manual]
Name: Description, dtype: object

In [9]:
# Number of invoices for each country
df.groupby(['Country'])['InvoiceNo'].count()

Country
Australia                 1259
Austria                    401
Bahrain                     19
Belgium                   2069
Brazil                      32
Canada                     151
Channel Islands            758
Cyprus                     622
Czech Republic              30
Denmark                    389
EIRE                      8196
European Community          61
Finland                    695
France                    8557
Germany                   9495
Greece                     146
Hong Kong                  288
Iceland                    182
Israel                     297
Italy                      803
Japan                      358
Lebanon                     45
Lithuania                   35
Malta                      127
Netherlands               2371
Norway                    1086
Poland                     341
Portugal                  1519
RSA                         58
Saudi Arabia                10
Singapore                  229
Spain                     2533


### **1.1 Observations:**

*From the data, we can see that:*

* The minimum price and quantity are negative, and this is impossible.
* There are null values in CustomerID and Description columns.
* Some Stock Codes are not a 5-digit integral number.
* Some Stock Codes has more than one item description.
* The number of customers is 4372.
* The number of countries is 38.

## **2. Data Preparation**

### This step includes Pre-Processing data before analyzing it.

### **2.1. Missing Data**

As we can see from the previous section, there are null data. 

So, I will remove the rows that contain null values.

In [10]:
# Delete rows with null CustomerID
clean_df = df.dropna()

# Check null values
clean_df.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [11]:
clean_df.shape

(406829, 8)

### **2.2. Dealing with Inconsistent Data**

*This section includes:*

* Removing the unspecified countries.
* Removing the price and quantity that are less than or equal to 0.
* Removing inconsistent Stock code.
* Keeping only one description for each Stock codes.

In [12]:
# Remove the unspecified countries
clean_df = clean_df[(clean_df.Country != 'Unspecified')]

In [13]:
# Removing the price and quantity that are less than 0
clean_df = clean_df[(clean_df.Quantity > 0) & (clean_df.UnitPrice >= 0)]

In [14]:
clean_df.describe()

,Quantity,UnitPrice,CustomerID
count,397680.000000,397680.000000,397680.000000
mean,13.025314,3.116123,15295.280185
std,180.475362,22.103411,1712.842626
min,1.000000,0.000000,12346.000000
25%,2.000000,1.250000,13969.000000
50%,6.000000,1.950000,15159.000000
75%,12.000000,3.750000,16800.000000
max,80995.000000,8142.750000,18287.000000


In [15]:
# Check the stock code

def has_right_scode(input):
    
    """
    Function: check the if the stock code is wirtten in a right way,
            The function check if the code contains 5-digit number or 5-digit number with a letter.
    Args:
      input(String): Stock code
    Return:
      Boolean: True or False
    """
    
    x = re.search("^\d{5}$", input)
    y = re.search("^\d{5}[a-zA-Z]{1}$", input)
    if (x or y):
        return True
    else:
        return False

    
clean_df['StockCode'] = clean_df['StockCode'].astype('str')
clean_df = clean_df[clean_df['StockCode'].apply(has_right_scode) == True]
clean_df.shape

(395835, 8)

In [16]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 395835 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    395835 non-null  object        
 1   StockCode    395835 non-null  object        
 2   Description  395835 non-null  object        
 3   Quantity     395835 non-null  int64         
 4   InvoiceDate  395835 non-null  datetime64[ns]
 5   UnitPrice    395835 non-null  float64       
 6   CustomerID   395835 non-null  float64       
 7   Country      395835 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.2+ MB


In [17]:
## One description for each stock code

# Put all Descriptions of each StockCode in a list 
df_items = pd.DataFrame(clean_df.groupby('StockCode')['Description'].unique()).reset_index()
df_items.head()

,StockCode,Description
0,10002,[INFLATABLE POLITICAL GLOBE ]
1,10080,[GROOVY CACTUS INFLATABLE]
2,10120,[DOGGY RUBBER]
3,10123C,[HEARTS WRAPPING TAPE ]
4,10124A,[SPOTS ON RED BOOKCOVER TAPE]


In [18]:
# StockCode that have more than one Description
df_items[df_items['Description'].str.len() != 1].head()

,StockCode,Description
41,16156L,"[WRAP, CAROUSEL, WRAP CAROUSEL]"
99,17107D,"[FLOWER FAIRY,5 SUMMER B'DRAW LINERS, FLOWER F..."
117,20622,"[VIPPASSPORT COVER , VIP PASSPORT COVER ]"
169,20725,"[LUNCH BAG RED RETROSPOT, LUNCH BAG RED SPOTTY]"
261,20914,"[SET/5 RED RETROSPOT LID GLASS BOWLS, SET/5 RE..."


***Note:*** As we can see, the items in the each Description list are the same item but they are written in different ways. So, I will keep only one description.

In [19]:
# Take one Description for each StockCode (*)
df_items.loc[:, 'Description'] = df_items.Description.map(lambda x: x[0])
df_items.head()

,StockCode,Description
0,10002,INFLATABLE POLITICAL GLOBE
1,10080,GROOVY CACTUS INFLATABLE
2,10120,DOGGY RUBBER
3,10123C,HEARTS WRAPPING TAPE
4,10124A,SPOTS ON RED BOOKCOVER TAPE


In [20]:
# Merge clean_df with df_items

clean_df = clean_df.drop('Description', axis = 1)
clean_df = pd.merge(clean_df, df_items, on = 'StockCode')

clean_df.head()

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,WHITE HANGING HEART T-LIGHT HOLDER
1,536373,85123A,6,2010-12-01 09:02:00,2.55,17850.0,United Kingdom,WHITE HANGING HEART T-LIGHT HOLDER
2,536375,85123A,6,2010-12-01 09:32:00,2.55,17850.0,United Kingdom,WHITE HANGING HEART T-LIGHT HOLDER
3,536390,85123A,64,2010-12-01 10:19:00,2.55,17511.0,United Kingdom,WHITE HANGING HEART T-LIGHT HOLDER
4,536394,85123A,32,2010-12-01 10:39:00,2.55,13408.0,United Kingdom,WHITE HANGING HEART T-LIGHT HOLDER


###***`Now, the data is ready to be used to analysis.`***

## **3. Data Analysis**

### This use the dataset to answer the questions, and discusses the results.

### **3.1 What is the preference product for the active customers?**

The active customers are the customers who buy from the store many times. These customers bought more times than the median of times of all customers. The preference product of the customer is the most purchased product by the customer.

In [21]:
# Count number of purchases for each customer
trans_num = pd.DataFrame(clean_df.groupby(['CustomerID'])['InvoiceNo'].count()).reset_index().rename(columns = {'InvoiceNo': 'Transactions'})
trans_num.head()

,CustomerID,Transactions
0,12346.0,1
1,12347.0,182
2,12348.0,27
3,12349.0,72
4,12350.0,16


In [22]:
# Calculate the median number of purchases
trans_median = round(trans_num["Transactions"].median())
trans_median

41

In [44]:
# Add the number of the transcations column to the clean data frame
act_cust = pd.merge(clean_df, trans_num, how = 'inner', on = 'CustomerID')

# Keep the customers that have number of transactions >= trans_median 
act_cust = act_cust[act_cust["Transactions"] >= trans_median]
act_cust.head()

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Description,Month,Transactions
0,536365,85123A,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,WHITE HANGING HEART T-LIGHT HOLDER,12,293
1,536373,85123A,6,2010-12-01 09:02:00,2.55,17850.0,United Kingdom,WHITE HANGING HEART T-LIGHT HOLDER,12,293
2,536375,85123A,6,2010-12-01 09:32:00,2.55,17850.0,United Kingdom,WHITE HANGING HEART T-LIGHT HOLDER,12,293
3,536396,85123A,6,2010-12-01 10:51:00,2.55,17850.0,United Kingdom,WHITE HANGING HEART T-LIGHT HOLDER,12,293
4,536406,85123A,8,2010-12-01 11:33:00,2.55,17850.0,United Kingdom,WHITE HANGING HEART T-LIGHT HOLDER,12,293


In [45]:
# Sum the quantity of each product based on the customer
act_cust = act_cust.groupby(['CustomerID', 'Description'])['Quantity'].sum().reset_index()
act_cust.head(10)

,CustomerID,Description,Quantity
0,12347.0,3D DOG PICTURE PLAYING CARDS,126
1,12347.0,3D SHEET OF CAT STICKERS,12
2,12347.0,3D SHEET OF DOG STICKERS,12
3,12347.0,60 TEATIME FAIRY CAKE CASES,72
4,12347.0,72 SWEETHEART FAIRY CAKE CASES,72
5,12347.0,AIRLINE BAG VINTAGE JET SET BROWN,28
6,12347.0,AIRLINE BAG VINTAGE JET SET RED,16
7,12347.0,AIRLINE BAG VINTAGE JET SET WHITE,12
8,12347.0,AIRLINE BAG VINTAGE TOKYO 78,32
9,12347.0,AIRLINE BAG VINTAGE WORLD CHAMPION,8


In [46]:
# Get the max quantity of products based on customer
act_cust = act_cust.groupby(['CustomerID']).max().reset_index()
act_cust.head()

,CustomerID,Description,Quantity
0,12347.0,WOODLAND DESIGN COTTON TOTE BAG,240
1,12349.0,ZINC FOLKART SLEIGH BELLS,36
2,12352.0,ZINC HEART FLOWER T-LIGHT HOLDER,36
3,12354.0,WOODEN HAPPY BIRTHDAY GARLAND,32
4,12356.0,WHITE HEART CONFETTI IN TUBE,168


In [26]:
# List of the preference products for each customer
prf_prod_cust = act_cust[['CustomerID', 'Description']]
prf_prod_cust.head()

,CustomerID,Description
0,12347.0,WOODLAND DESIGN COTTON TOTE BAG
1,12349.0,ZINC FOLKART SLEIGH BELLS
2,12352.0,ZINC HEART FLOWER T-LIGHT HOLDER
3,12354.0,WOODEN HAPPY BIRTHDAY GARLAND
4,12356.0,WHITE HEART CONFETTI IN TUBE


In [27]:
# Top 20 favorite products for the customers
prf_prod_cust.groupby(['Description']).count().sort_values('CustomerID', ascending=False).head(20)

,CustomerID
Description,
ZINC METAL HEART DECORATION,163
ZINC T-LIGHT HOLDER STARS SMALL,120
ZINC FOLKART SLEIGH BELLS,108
ZINC WILLIE WINKIE CANDLE STICK,108
YOU'RE CONFUSING ME METAL SIGN,91
ZINC HERB GARDEN CONTAINER,78
WORLD WAR 2 GLIDERS ASSTD DESIGNS,74
WRAP VINTAGE PETALS DESIGN,70
WOODLAND CHARLOTTE BAG,44


### **3.2 What is the top three preference products in each country?**

The preference product in a country is the most purchased product in that country.

In [28]:
# Sum the quantity of each product based on the country
prf_prod_country = clean_df.groupby(['Country', 'Description'])['Quantity'].sum().reset_index()
prf_prod_country.head()

,Country,Description,Quantity
0,Australia,DOLLY GIRL BEAKER,200
1,Australia,I LOVE LONDON MINI BACKPACK,4
2,Australia,10 COLOUR SPACEBOY PEN,48
3,Australia,12 PENCIL SMALL TUBE WOODLAND,384
4,Australia,12 PENCILS TALL TUBE POSY,252


In [29]:
# Top 3 favorite products in each country
prf_prod_country = prf_prod_country.set_index('Description').groupby('Country')['Quantity'].nlargest(3).reset_index()
prf_prod_country = prf_prod_country[['Country', 'Description']]
prf_prod_country.head(15)

,Country,Description
0,Australia,MINI PAINT SET VINTAGE
1,Australia,RABBIT NIGHT LIGHT
2,Australia,RED HARMONICA IN BOX
3,Austria,SET 12 KIDS COLOUR CHALK STICKS
4,Austria,MINI JIGSAW PURDEY
5,Austria,PACK OF 6 PANNETONE GIFT BOXES
6,Bahrain,ICE CREAM SUNDAE LIP GLOSS
7,Bahrain,DOUGHNUT LIP GLOSS
8,Bahrain,GROW A FLYTRAP OR SUNFLOWER IN TIN
9,Belgium,PACK OF 72 RETROSPOT CAKE CASES


### **3.3 What is the top three preference products in each month?**

The preference product in a month is the most purchased product in that month.

In [32]:
# Create month column from the InvoiveDate column 
clean_df['Month'] = pd.DatetimeIndex(clean_df['InvoiceDate']).month
clean_df.head()

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Description,Month
0,536365,85123A,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,WHITE HANGING HEART T-LIGHT HOLDER,12
1,536373,85123A,6,2010-12-01 09:02:00,2.55,17850.0,United Kingdom,WHITE HANGING HEART T-LIGHT HOLDER,12
2,536375,85123A,6,2010-12-01 09:32:00,2.55,17850.0,United Kingdom,WHITE HANGING HEART T-LIGHT HOLDER,12
3,536390,85123A,64,2010-12-01 10:19:00,2.55,17511.0,United Kingdom,WHITE HANGING HEART T-LIGHT HOLDER,12
4,536394,85123A,32,2010-12-01 10:39:00,2.55,13408.0,United Kingdom,WHITE HANGING HEART T-LIGHT HOLDER,12


In [60]:
# Sum the quantity of each product based on the month
prf_prod_mnth = clean_df.groupby(['Month','Description'])['Quantity'].sum().reset_index()
prf_prod_mnth.head()

,Month,Description,Quantity
0,1,4 PURPLE FLOCK DINNER CANDLES,2
1,1,OVAL WALL MIRROR DIAMANTE,12
2,1,SET 2 TEA TOWELS I LOVE LONDON,417
3,1,10 COLOUR SPACEBOY PEN,276
4,1,12 COLOURED PARTY BALLOONS,80


In [61]:
# Top 3 favorite products in each Month
prf_prod_mnthh = prf_prod_mnth.set_index('Description').groupby(['Month'])['Quantity'].nlargest(3).reset_index()
prf_prod_mnthh

,Month,Description,Quantity
0,1,MEDIUM CERAMIC TOP STORAGE JAR,74215
1,1,WHITE HANGING HEART T-LIGHT HOLDER,5467
2,1,PACK OF 72 RETROSPOT CAKE CASES,3329
3,2,EMPIRE DESIGN ROSETTE,3986
4,2,GROW A FLYTRAP OR SUNFLOWER IN TIN,3795
5,2,WORLD WAR 2 GLIDERS ASSTD DESIGNS,3457
6,3,JUMBO BAG RED RETROSPOT,4924
7,3,WORLD WAR 2 GLIDERS ASSTD DESIGNS,3888
8,3,PACK OF 72 RETROSPOT CAKE CASES,3720
9,4,WORLD WAR 2 GLIDERS ASSTD DESIGNS,10224


In [75]:
# The favorite products in more than one month
prf_prod_mnths = prf_prod_mnthh.groupby('Description')['Month'].count().to_frame()
prf_prod_mnths[prf_prod_mnths['Month']>1].reset_index()

,Description,Month
0,ASSORTED COLOURS SILK FAN,2
1,JUMBO BAG RED RETROSPOT,4
2,PACK OF 72 RETROSPOT CAKE CASES,2
3,RABBIT NIGHT LIGHT,2
4,SMALL POPCORN HOLDER,6
5,WHITE HANGING HEART T-LIGHT HOLDER,2
6,WORLD WAR 2 GLIDERS ASSTD DESIGNS,7


# **SUMMARY**

### ***From above, we can see that the preference of products in an online store are different based on the customer, country and month of purchase. So, to make good products recommendations we have to take into consideration these factors.***

### ***Analyzing e-commerce transactions can help us in identifying the most frequently purchased products by the customer, and then make good products recommendations to the customers. Taking into consideration the customer country and the month of purchase can help in give better recommendations.***